In [276]:
import pandas as pd

In [277]:
data = pd.read_csv('IOT-temp.csv', index_col="id")
data.head()

,room_id/id,noted_date,temp,out/in
id,,,,
__export__.temp_log_196134_bd201015,Room Admin,08-12-2018 09:30,29,In
__export__.temp_log_196131_7bca51bc,Room Admin,08-12-2018 09:30,29,In
__export__.temp_log_196127_522915e3,Room Admin,08-12-2018 09:29,41,Out
__export__.temp_log_196128_be0919cf,Room Admin,08-12-2018 09:29,41,Out
__export__.temp_log_196126_d30b72fb,Room Admin,08-12-2018 09:29,31,In


In [278]:
print(len(data))

97606


In [279]:
print(data.isnull().sum())

room_id/id    0
noted_date    0
temp          0
out/in        0
dtype: int64


In [280]:
data_1 = data.loc["__export__.temp_log_196134_bd201015":"__export__.temp_log_60240_9177689a"] 
print(len(data_1))
data_1.to_csv('batch_1.csv')

32535


In [281]:
data_2 = data.loc["__export__.temp_log_60239_4eb9e90b":"__export__.temp_log_12544_1a76a951"] 
print(len(data_2))
data_2.to_csv('batch_2.csv')

32535


In [282]:
data_3 = data.loc["__export__.temp_log_148002_0a1a4e64":"__export__.temp_log_133741_32958703"] 
print(len(data_3))
data_3.to_csv('batch_3.csv')

32536


In [283]:
for i in [{"title": "In", "boolean": "In"}, {"title": "Out", "boolean": "Out"}]:
    print(f"{i['title']}: {(data_3['out/in'] == i['boolean']).sum()}")

In: 10648
Out: 21888


In [284]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import col, unix_timestamp, from_unixtime, hour, when
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("RandomForestClassifier").getOrCreate()

# Load data
data = spark.read.csv("batch_2.csv")
data = data.filter(data._c0 != 'id')
data.limit(5).show()

+--------------------+----------+----------------+---+---+
|                 _c0|       _c1|             _c2|_c3|_c4|
+--------------------+----------+----------------+---+---+
|__export__.temp_l...|Room Admin|17-10-2018 20:37| 39|Out|
|__export__.temp_l...|Room Admin|17-10-2018 20:37| 39|Out|
|__export__.temp_l...|Room Admin|17-10-2018 20:37| 39|Out|
|__export__.temp_l...|Room Admin|17-10-2018 20:37| 39|Out|
|__export__.temp_l...|Room Admin|17-10-2018 20:37| 39|Out|
+--------------------+----------+----------------+---+---+



In [285]:
data = data.withColumn("_c5", when(data["_c4"] == "Out", 1).otherwise(0))
data = data.drop("_c0", "_c1", "_c4")
data.limit(5).show()

+----------------+---+---+
|             _c2|_c3|_c5|
+----------------+---+---+
|17-10-2018 20:37| 39|  1|
|17-10-2018 20:37| 39|  1|
|17-10-2018 20:37| 39|  1|
|17-10-2018 20:37| 39|  1|
|17-10-2018 20:37| 39|  1|
+----------------+---+---+



In [286]:
data = data.withColumn("_c2", from_unixtime(unix_timestamp("_c2", "dd-MM-yyyy HH:mm")))
data = data.withColumn("_c2", 
    when(hour(col("_c2")) < 12, "pagi")
    .when((hour(col("_c2")) >= 12) & (hour(col("_c2")) < 18), "siang")
    .otherwise("malam"))

data.limit(5).show()

+-----+---+---+
|  _c2|_c3|_c5|
+-----+---+---+
|malam| 39|  1|
|malam| 39|  1|
|malam| 39|  1|
|malam| 39|  1|
|malam| 39|  1|
+-----+---+---+



In [287]:
data.select("_c2").distinct().show()

# Initialize StringIndexer (temporary output column name to avoid conflict)
indexer = StringIndexer(inputCol="_c2", outputCol="_c2_indexed")

# Apply the transformation to create the new indexed column
data = indexer.fit(data).transform(data)

# Drop the old _c2 column
data = data.drop("_c2")

# Rename _c2_indexed to _c2 to replace the old column
data = data.withColumnRenamed("_c2_indexed", "_c2")

# Convert _c3 to numeric type (e.g., IntegerType or DoubleType)
data = data.withColumn("_c3", col("_c3").cast("double"))

assembler = VectorAssembler(inputCols=["_c3", "_c5"], outputCol="features")
data = assembler.transform(data).select("features", "_c2")

data.limit(5).show()

+-----+
|  _c2|
+-----+
|siang|
|malam|
| pagi|
+-----+

+----------+---+
|  features|_c2|
+----------+---+
|[39.0,1.0]|2.0|
|[39.0,1.0]|2.0|
|[39.0,1.0]|2.0|
|[39.0,1.0]|2.0|
|[39.0,1.0]|2.0|
+----------+---+



In [288]:
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

In [289]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Initialize the RandomForestClassifier with the best hyperparameters
rf = RandomForestClassifier(labelCol="_c2", featuresCol="features", 
                            numTrees=50, maxDepth=18, maxBins=128)

# Define the parameter grid (you can add more values to experiment with)
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [50]).addGrid(rf.maxDepth, [18]).addGrid(rf.maxBins, [128]).build()

In [290]:
# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="_c2", metricName="accuracy")

# Set up CrossValidator with the updated paramGrid
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Run cross-validation and choose the best set of parameters
cvModel = crossval.fit(train_data)

In [291]:
predictions = cvModel.transform(test_data)

In [292]:
evaluator = MulticlassClassificationEvaluator(labelCol="_c2", predictionCol="prediction", metricName="accuracy")
roc_auc = evaluator.evaluate(predictions)

print(f"accuracy: {roc_auc}")

accuracy: 0.6457945856581938


In [293]:
predictions.select("features", "_c2", "prediction").show(5)

+----------+---+----------+
|  features|_c2|prediction|
+----------+---+----------+
|[22.0,0.0]|1.0|       0.0|
|[25.0,0.0]|0.0|       0.0|
|[25.0,0.0]|0.0|       0.0|
|[25.0,0.0]|0.0|       0.0|
|[25.0,0.0]|2.0|       0.0|
+----------+---+----------+
only showing top 5 rows



In [294]:
for i in [{"title": "pagi", "label": 0}, {"title": "malam", "label": 1}, {"title": "siang", "label": 2}]:
    print(f"{i['title']}: {predictions.filter(predictions['prediction'] == i['label']).count()}")

pagi: 5341
malam: 3903
siang: 434


In [295]:
predictions = predictions.withColumn("prediction", when(predictions["prediction"] == 0, "pagi").when(col("prediction") == 1, "malam").otherwise("siang"))
predictions.select("features", "_c2", "prediction").show(5)

+----------+---+----------+
|  features|_c2|prediction|
+----------+---+----------+
|[22.0,0.0]|1.0|      pagi|
|[25.0,0.0]|0.0|      pagi|
|[25.0,0.0]|0.0|      pagi|
|[25.0,0.0]|0.0|      pagi|
|[25.0,0.0]|2.0|      pagi|
+----------+---+----------+
only showing top 5 rows



In [296]:
# Convert to Pandas DataFrame
pandas_df = predictions.select("features", "_c2", "prediction").toPandas()

pandas_df.to_csv("result_from_train.csv", index=False)

In [ ]:
import json
from flask import Flask, jsonify, request
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, unix_timestamp, col
from pyspark.ml.feature import VectorAssembler
import os

app = Flask(__name__)

UPLOAD_FOLDER = os.getcwd()  # Current directory
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Initialize Spark session
spark = SparkSession.builder.appName('FileProcessingApp').getOrCreate()

@app.route('/api_2', methods=['POST'])
def create():
    try:
        # Access the uploaded file and get its filename
        file = request.files['file']
        file_name = file.filename

        # Extract the pure file name (without any path)
        pure_file_name = os.path.basename(file_name)

        # Define the path where the file will be saved (current folder)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], pure_file_name)

        print(pure_file_name)
        
        # Save the file to the current folder
        file.save(file_path)

        # Read the CSV file into a Spark DataFrame
        spark_df = spark.read.csv(pure_file_name)
        spark_df = spark_df.filter(spark_df["_c0"] != 'id')
        spark_df = spark_df.withColumn("_c5", when(spark_df["_c4"] == "Out", 1).otherwise(0))
        spark_df = spark_df.drop("_c0", "_c1", "_c4", "_c2")

        # Convert _c3 to numeric type (e.g., IntegerType or DoubleType)
        spark_df = spark_df.withColumn("_c3", col("_c3").cast("double"))

        assembler = VectorAssembler(inputCols=["_c3", "_c5"], outputCol="features")
        # spark_df = assembler.transform(spark_df).select("features", "_c2")
        spark_df = assembler.transform(spark_df).select("features")

        predictions = cvModel.transform(spark_df)

        # Export predictions to pandas DataFrame
        export = predictions.drop("rawPrediction", "probability").withColumn("prediction", when(predictions["prediction"] == 0, "pagi").when(col("prediction") == 1, "malam").otherwise("siang")).toPandas()

        # Save the results to a CSV file
        export.to_csv("result_from_api.csv", index=False)
        
        return jsonify({"message": "File successfully saved as result_from_api.csv"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
   app.run(port=5000)